# Задача восстановления пропущенных пробелов

Для решения использовалась предобученная модель `UrukHan/t5-russian-spell` (https://huggingface.co/UrukHan/t5-russian-spell) из библиотеки Hugging Face. Эта модель основана на архитектуре T5 (Text-to-Text Transfer Transformer) и обучена специально для русского языка на задачах исправления опечаток.

Инференс выполнялся локально на GPU NVIDIA T4, доступной в Google Colab. Это обеспечило высокую скорость предсказаний и возможность использования beam search для повышения качества результатов.

Настройка модели включала следующие параметры:
- `num_beams=5` — используется beam search, чтобы выбирать наиболее вероятный вариант разделения слов
- `do_sample=False` — детерминированная генерация без случайного семплирования
- `max_length=len(text)*2` — максимальная длина предсказанного текста, чтобы корректно обрабатывать длинные строки
- `early_stopping=True` — генерация останавливается, когда модель достигает оптимального результата


In [1]:
!pip install -q transformers accelerate sentencepiece

In [72]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

# Загружаем предобученную модель для русского языка, которая умеет исправлять ошибки и вставлять пробелы
model_name = "UrukHan/t5-russian-spell"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

# Функция, которая вставляет пробелы в слитный текст с помощью модели
def insert_spaces_spell(text):
    prompt = f"Раздели пробелами следующий текст: {text}."
    out = pipe(prompt, max_length= len(text)*2, num_beams=5, do_sample=False)[0]
    res = out.get('generated_text') or out.get('text') or ""
    return res.strip()

Device set to use cuda:0


## Получение позиций пробелов
После предсказания текста с пробелами выполняется преобразование в позиции пробелов с помощью функции `spaced_to_positions`, которая проходит по сгенерированному тексту и записывает индекс каждого пробела.

In [68]:
# Функция для преобразования текста с пробелами в позиции пропущенных пробелов
def spaced_to_positions(orig_no_space, pred_with_spaces):
    s0 = str(orig_no_space).replace(" ", "").strip()
    pred = str(pred_with_spaces).replace(" ", "").strip()

    positions = []
    j = 0

    for idx, ch in enumerate(pred_with_spaces):
        if ch == " ":
            positions.append(j)
        else:
            j += 1
    return positions


In [69]:
# Функция, возвращающая ответ в виде строки
def predict_positions(text):
    pred = insert_spaces_spell(text)
    positions = spaced_to_positions(text, pred)
    return str(positions) if positions else "[]"

In [70]:
import pandas as pd

rows = []
# Чтение данных из файла
# Чтение производится по строкам, т.к. во втором столбце могут встречаться запятые
with open("dataset_1937770_3.txt", "r", encoding="utf-8") as f:
    next(f)
    for line in f:
        line = line.strip()
        if not line:
            continue
        id_, text = line.split(",", 1)
        rows.append((id_, text))


df = pd.DataFrame(rows, columns=["id", "text_no_spaces"])
print(df.head())

  id                 text_no_spaces
0  0                куплюайфон14про
1  1             ищудомвПодмосковье
2  2  сдаюквартирусмебельюитехникой
3  3     новыйдивандоставканедорого
4  4                отдамдаромкошку


In [76]:
# Пример работы на маленьком подмножестве
subset_df = df.iloc[:15].copy()

subset_df['predicted_positions'] = subset_df['text_no_spaces'].apply(predict_positions)

In [78]:
subset_df.head()

,id,text_no_spaces,predicted_positions
0,0,куплюайфон14про,[]
1,1,ищудомвПодмосковье,"[3, 6, 7]"
2,2,сдаюквартирусмебельюитехникой,"[4, 12, 13, 20, 21]"
3,3,новыйдивандоставканедорого,"[5, 18]"
4,4,отдамдаромкошку,"[5, 10]"


In [79]:
# Предсказание для всего набора данных
df['predicted_positions'] = df['text_no_spaces'].apply(predict_positions)
df.to_csv('submission.csv', index=False)

Этот метод решения давал лучшие результаты, но также был реализован алгоритм динамического программирования с использованием информации только о частотах.
Для алгоритма с использованием модели из Huggingface f1-мера получилась равной 64.171% , а для алгоритма динамического программирования 44.838%. Ссылка на решение с более быстрой, но худшей по качеству программой: https://github.com/nlpgirl/avito_staj/blob/main/dynamic_programming.ipynb